## Annotation via BiomaRt with Functional Enrichment analysis


**Today, this notebook constitutes your in-class activity.**

We will pick up from where we left off with the analysis of our gene expression data set from last time.

We left off having performed our differential gene\-expression analysis. However, our results were in terms of ensembl gene IDs rather than human\-readable gene symbols; we'd like to be able to know what genes are actually of interest in our analysis. Today, we'll connect our analysis to annotation data so that we can obtain this information.



**Q1.** Let's start by load the following packages into R:

- The annotation library package `biomaRt`
- Our new favoriate library package `tidyverse`

**Provide and execute your R code below.**

Let's recover the differential expression from the last RNA\-seq module. 

**Q2.** Next:

- Load the results file (named `ensembl_id_DE_results.csv`) from the previous analysis into your notebook to a variable called `de_results`. You can use `read.csv()` or `read.table(sep=',')`, just make sure you parse the file correctly!
- Then, use `as_tibble()` and that as a tidyverse table in `de_results_tbl`. 

**Provide and execute your R code below.**


**Q3.** OK, now let's obtain some annotation information.

- Using `useEnsembl()`, load the appropriate library from the ensembl database for humans, and save in a variable named `mymart_human`. (Hint: `hsapiens_gene_ensembl` is the database you are after!)

**Provide and execute your R code below.**

**Q4.** Now, using the mart that you just created (`mymart_human`), create an annotation table which includes the `ensembl_gene_id` as well as a gene symbol for each transcript in the results data frame.

- use `getBM()` to create to extract the attributes: 
```
    ensembl_gene_id
    external_gene_name
```

As values, use the ensembl ids you have present in `de_results_tbl`.
- Save this in an output variable named `lookup`. 

**Provide and execute your R code below.**

**Q5.** Building upon that command, let's get some additional annotation information. Note that getting all of the annotations will take a little bit of time! 

Using `listAttributes(mymart_human)` will show what attributes that you can extract.

- obtain the following annotations from ensembl stored in a new variable called `annot` for your list of ensembl gene ids:
  ```
    external_gene_name/n    chromosome name
    start position
    end position
    strand
    Gene % GC content
    external gene name
    description
  ```

- Then, output to your notebook the first 5 entries in your table

**Provide and execute your R code below.**

**Q6.** Output your annotations to file that reports everything but the description, without quotes, and separated by commas (useful for data parsing) to a file called `annot-forparse.csv`

**Provide and execute your R code below.**


**Q7.** Now that we have annotations, we need to merge them together with the table that contained our association information. However, notice that the annot matrix lists the ensembl ids in a different order than the statistical association table.

- Using one of the tidyverse `*_join()` family of functions, merge the `annot` frame with `de_results_tbl`. 

TIP: Be sure choose the join function with keeps *all* the entries in your result table `de_results_tbl`, and not extra entries that may be present in `annot`!

- save this to a new table called `de_results_annot_tbl`.

**Provide and execute your R code below.**

**Q8.** Let's sort the table to get the most significant results to the top of the table. 

- Arrange the the table by the column `pvalue`. (Hint: `order()` in base R or `arrange()` in tidyverse might be helpful!)
- Store the sorted table to a new variable called `de_results_annot_srt_tbl`. 
- output the results of first 10 rows of `de_results_annot_sort_tbl`

**Provide and execute your R code below.**  

**Q9.** What are the genes symbols that you found for the top 10 results? 

**Q10.**  In the majority of cases, our transcripts will have gene symbols attached to them. However, sometimes this will not happen. Let's find out more about what those missing ones turn out to be in some cases. 

How many entries in our results have either missing `""` or `NA` results for their `external_gene_name`?

- use `group_by()`, `summarize()`, `arrange()` to obtain the counts of each gene symbol present.
- Hint: you may want to arrange by descending order to get the biggest entries at the top of your report; thus embedding `desc()` within the invocation of `arrange()` may be helpful here.

**Provide and execute your R code below.** 

**Q11.** What are the top 3 differentially expressed transcripts (in terms of p-value) where `external_gene_name` is missing `""`? 

Go to `ensembl.org` and search for each of these transcripts. For each, are there plausible explanations for why this is missing? 

In particular, what does the annotation "TEC" mean?

- Hint: `filter()` may by of use to you here.

**Q12.** Next, let's curate a final table where missing gene symbols (`""`), gene symbols that are `'NA'`, and duplicate entries are removed. To do that:

- Using `de_results_annot_srt_table`, use `filter()` to remove genes where `external_gene_name` is "" or `'NA'`. Save this result to `de_results_no_missing`
- Use `duplicated()` in combination with `de_results_no_missing` to find the indexes of duplicated external gene names
- Remove the duplicated external gene names and save the output to a variable named `de_results_annot_srt_filt_tbl`

**Provide and execute your R code below.**

**Q13.** To get prepared for enrichment analysis, first need the list of all genes we tested (the background). To do that:

- use `write.table()` to print out the list of genes from `de_results_annot_srt_filt_tbl`, setting `quote=F`, `row.names=F`,  `col.names=F`. Name the file `background_genelist.txt`.
- use `write.table()` to print out associated genes with pvalue < 0.001 from `de_results_annot_srt_filt_tbl`, setting `quote=F`, `row.names=F`, `col.names=F`. Name the file `testset_genelist.txt`.

**Provide and execute your R code below.**

#### Now, let us use [WebGestalt](http://www.webgestalt.org/) to perform some ontology analysis using the unique set of genes created above.

- Click "ORA Sample Run". Under "Basic Parameters" choose the appropriate organism, Over-Representation Analysis (ORA) as the method, and geneontology -> Biological_process_noRedundant as the functional database.
- In the Gene List section, select Gene ID type as "Gene symbol". Reset the "Upload Gene List", click on "click to upload" and upload `testset_genelist.txt` here.
- In the Reference Gene List section, Click Reset, and click to upload a user reference set, and upload `background_genelist.txt`. Set the Type of reference set to "gene Symbol".
- Under "Advanced parameters", make sure BH (Benjamini-Hochberg) is selected for multiple test correction, set the FDR to 0.05. Leave the other options unchanged.
- Hit submit.  

**Q12.** What background gene list did you select? Why is this important?   



**Q13.** Explore the output of the analysis. Does that tool find all of the genes that you provided? Why or why not? How could you fix this?

**Q14.** Do any of the associations pass a FDR (false discovery rate) of 0.002 or better? What does "significance level" mean? If yes, do any of these categories make plausible or intuitive sense for the gene expression analysis that you performed?

**Q15.** Here, we selected genes with nominal p < 0.001 for enrichment analysis. Was this choice warrented? Why or why not? Having performed this analysis, what might you do differently?

**Q16.** One final item in your pipeline to finish it off: run the R function `sessionInfo()` to record the functions and versions used for your analysis.

**Provide and execute your R code below.**

### Congrats! You have completed your first pipeline! High five yourself and breathe a sigh of relief. :)